In [1]:
%load_ext watermark

In [2]:
import pandas as pd
import itertools as itt
import numpy as np
import importlib as imp
from pathlib import Path
import sqlite3 as sq
import altair as alt

import sys
sys.path.append("../")

DB_PATH = '../data/data.sqlite.db'
VIZ_PATH = '../vis/'


%watermark --iversions

sqlite3: 2.6.0
numpy  : 1.22.3
pandas : 1.4.2
altair : 4.2.0
sys    : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:25:59) 
[GCC 10.3.0]



In [3]:
with sq.connect(DB_PATH) as conn:
        sql = "select `start-year`, AVG(`home-points` - `away-points`) as `home-advantage` from games JOIN seasons ON seasons.id = games.season_id GROUP BY `start-year`;"        
        adv_df = pd.read_sql_query(sql, conn)
        cur = conn.cursor()
        cur.execute("select AVG(`home-points` - `away-points`) from games")
        diff = cur.fetchone()        
        print("Total advantage {:.3f}".format(diff[0]))

Total advantage 3.793


In [4]:
adv_df['rolling-advantage'] = adv_df['home-advantage'].rolling(3).mean()

In [5]:
c1 = alt.Chart(adv_df).mark_bar().encode(
    x=alt.X("start-year:O", title="Start year of the season"),
    y=alt.Y("home-advantage", title="Home court advantage", axis=alt.Axis(grid=False), scale=alt.Scale(domain=(-2, 10)))
)

c2 = alt.Chart(adv_df).mark_line(color="red").encode(
    x=alt.X("start-year:O", title="Start year of the season"),
    y=alt.Y("rolling-advantage", title="Home court advantage", axis=alt.Axis(grid=False), scale=alt.Scale(domain=(-2, 10)))
)

c = (c1 + c2).properties(width=500, height=200)
c

alt.LayerChart(...)